# Stage directions: basic features

This notebook contains:

* directions extraction
* basic values calculation

## Imports
<span style="font-size:larger;color:blue;">Run this before everything else!</span>

In [1]:
# imports
import os
import re
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
from statistics import mean
from pymystem3 import Mystem


# we need this namespaces parameter to search in TEI-encoded files correctly
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

# one Mystem for everything, so that we won't have to initialize it every time
mystem = Mystem()

## Preparation

### Data
I used [RusDraCor](https://github.com/dracor-org/rusdracor) as of 9th April, 2018. More precise data on the corpus is below.

In [2]:
corpus_path = ".." + os.sep + "RusDraCor"

### Play class
I want to create a table with information on the plays we research, so it will be much easier to have a separate ```Play``` class with all the data.


```python
class Play:
    def __init__(self, path, author, title, year_written, year_published, amount_of_directions, amount_of_acts):
        self.path = path
        self.author = author
        self.title = title
        self.year_written = year_written
        self.year_published = year_published
        self.amount_of_directions = amount_of_directions
        self.amount_of_acts = amount_of_acts
```

Update: we don't really need that one.

### Folders and files

In [3]:
directions_folder = ".." + os.sep + "directions"
if not os.path.exists(directions_folder):
    os.mkdir(directions_folder)


csv_folder = "." + os.sep + "csv"
if not os.path.exists(csv_folder):
    os.mkdir(csv_folder)
    

all_directions_file = directions_folder + os.sep + "all_directions.txt"
if not os.path.exists(all_directions_file):
    f = open(all_directions_file, "w", encoding="utf-8")
    f.close()

## Extraction

### Dates issue

There's a certain problem with the date of the play: the corpus provides us three various dates, which are `print`, `premiere` and `written`. Sometimes not all of them are present. To get a single date for a play, we do the following:

1. If `print` and `premiere` are available, we take the minimum of the two. 
2. If `written` is more than 10 years before `print` or `premiere`, we take `written`. This is the case when the play was censored, banned, or probably discovered after author's death. By that we achieve that a manuscript of a play that was only printed 10 years or later after its inception will be grouped among plays from a different time.

In [4]:
def single_date(date_print, date_premiere, date_written):
    if date_print and date_premiere:
        date_definite = min(date_print, date_premiere)
    elif date_premiere:
        date_definite = date_premiere
    else:
        date_definite = date_print

    if date_written and date_definite:
        if date_definite - date_written > 10:
            date_definite = date_written
        elif date_written and not date_definite:
            date_definite = date_written
    return date_definite

### Play information
The majority of information pieces can be extracted from the ```html <teiHeader></teiHeader>``` in XML itself. Each act is wrapped in ```<div type="act">```, so counting the acts also isn't much of a problem.

In [5]:
def get_play_data(play_path):
    reg_space = re.compile("\s{2,}")
    root = etree.parse(play_path)
    
    title = root.find(".//tei:titleStmt/tei:title", tei_ns).text
    title = reg_space.sub(" ", title)
    
    author = root.find(".//tei:titleStmt/tei:author", tei_ns).text
    
    try:
        written_str = root.find(".//tei:sourceDesc/tei:bibl/tei:bibl/tei:date[@type=\"written\"]", tei_ns).attrib["when"]
        year_written = int(written_str)
    except:
        year_written = None
    try:
        published_str = root.find(".//tei:sourceDesc/tei:bibl/tei:bibl/tei:date[@type=\"print\"]", tei_ns).attrib["when"]
        year_published = int(published_str)
    except:
        year_published = None
    try:
        premiere_str = root.find(".//tei:sourceDesc/tei:bibl/tei:bibl/tei:date[@type=\"premiere\"]", tei_ns).attrib["when"]
        year_premiere = int(premiere_str)
    except:
        year_premiere = None
    year_definite = single_date(year_published, year_premiere, year_written)
        
    acts = len(root.findall(".//tei:text/tei:body/tei:div[@type=\"act\"]", tei_ns))
    return author, title, year_definite, acts    

### Amount of words in a play
Word count is required to count some averages later on.

In [6]:
def word_count(play_path, mystem):
    with open(play_path, "r", encoding="utf-8") as play:
        soup = BeautifulSoup(play, "xml")
        text = soup.find("text").text
    lemmas_play = set(mystem.lemmatize(text))
    words_play = [item for item in mystem.analyze(text) if "analysis" in item]
    return len(lemmas_play), len(words_play)

### Directions
Basically, all the directions are wrapped by the ```<stage></stage>``` tag, so we can just use regex. The only problem with this approach is that sometimes directions are on separate lines, and many spaces and tabs may occur in between two words — and this can also be fixed with the help of regex. 

In [7]:
def get_directions(play_path):
    reg_space = re.compile("\s{2,}")
    root = etree.parse(play_path)
    directions_tags = root.findall(".//tei:stage", tei_ns)
    directions_text = []
    for direction in directions_tags:
        try:
            text = direction.text
            text = reg_space.sub(" ", text)
            text = text.replace("\xa0", " ")
            text = text.strip("\(\) \.")
            text = text.lower()
            directions_text.append(text)
        except:
            pass
    return directions_text

In addition — let's count the average length of a direction in a play!

In [8]:
def average_direction(directions, mystem):
    dir_length = []
    dir_lemmas = []
    for direction in directions:
        length = len([item for item in mystem.analyze(direction) if "analysis" in item])
        dir_length.append(length)
        lemmas = len(set(mystem.lemmatize(direction)))
        dir_lemmas.append(lemmas)
    return mean(dir_length), mean(dir_lemmas)

## Putting it all together

### Saving the data
The following is being done in order to save the extracted directions just in case (and not to extract them from the play every time we need them).

In [9]:
def save_directions(play_name, directions, option):
    if option == "sep":
        directions_file = directions_folder + os.sep + play_name.replace("xml", "txt")
        with open(directions_file, "w", encoding="utf-8") as directions_save:
            directions_save.write("\n".join(directions))
    elif option == "all":
        with open(all_directions_file, "a", encoding="utf-8") as all_directions_save:
            all_directions_save.write("\n".join(directions) + "\n")

### Working with the corpus

Now, we traverse all the plays in the ```corpus_folder``` and do the following: 
* extract all the directions and save them: 
    * in a ```directions_list``` variable,
    * in a separate file,
    * in a file that contains all the directions
* extract information about a play and save it into ```play_list``` (to convert it into a Pandas dataframe later)

In [10]:
# this is required because I'm using a Mac, which sometimes creates system folders like .DS_Store;
# in any other cases -- never mind
play_files = [item for item in os.listdir(corpus_path) if item.endswith(".xml")]

In [11]:
directions_list = []
play_list = []
dropped_count = 0

for play in play_files:
    play_path = corpus_path + os.sep + play
    
    st_dirs = get_directions(play_path)
    author, title, year, acts = get_play_data(play_path)
    lemmas, words = word_count(play_path, mystem)
    words_dir, lemmas_dir = average_direction(st_dirs, mystem)
    
    play_info = {"File": play, 
            "Author": author, 
            "Title": title, 
            "Year": year,
            "Amount of directions": len(st_dirs), 
            "Amount of acts": acts if acts!=0 else 1,
            "Lemmas": lemmas,
            "Words": words,
            "Words per direction": words_dir,
            "Lemmas per direction": lemmas_dir}
    
    if play_info["Year"]:
        # directions
        directions_list.append(st_dirs)
        save_directions(play, st_dirs, option="sep")
        save_directions(play, st_dirs, option="all")
        
        # add information about a play
        play_list.append(play_info)
    else:
        dropped_count += 1
        print("Dropped a play: {}".format(play_path))

print("All in all, dropped {} plays.".format(dropped_count))

All in all, dropped 0 plays.


#### Corrections
Note that the first ```if``` statement in the ```for``` loop removes all the plays in which __the `year` is not clear__. Luckily, we don't have those :)

Furthermore, if there were no ```<div type="act">```, it means that __there is only one act in the play (not zero)__. I corrected it while creating ```play_info```.

### Data frame

Now, we create a Pandas data frame with all the information about the plays. We'll use paths to the plays to refer to rows.

In [12]:
play_df = pd.DataFrame(play_list)
play_df.head()

,Amount of acts,Amount of directions,Author,File,Lemmas,Lemmas per direction,Title,Words,Words per direction,Year
0,1,38,"Пушкин, Александр Сергеевич",pushkin-stseny-iz-rytsarskih-vremen.xml,996,4.973684,Сцены из рыцарских времен,3399,3.131579,1837
1,1,687,"Тургенев, Иван Сергеевич",turgenev-holostjak.xml,2785,6.391557,Холостяк,21501,4.506550,1849
2,2,254,"Гоголь, Николай Васильевич",gogol-zhenitba.xml,2187,5.952756,Женитьба,13094,3.925197,1842
3,3,88,"Крылов, Иван Андреевич",krylov-sonnyj-poroshok-ili-pohischennaja-krest...,1550,6.522727,Сонный порошок или похищенная крестьянка,7605,4.215909,1800
4,1,80,"Блок, Александр Александрович",blok-neznakomka.xml,1342,18.025000,Незнакомка,4222,16.462500,1907


## Simple numbers
We will calculate:

* word and lemma count,
* amounts of the following parts-of-speech:
    - nouns,
    - adjectives,
    - verbs,
    - adverbs,
    - interjections
* amount of directions, per act

### Word, lemma count
Word and lemma count were calculated previously (see "Extraction" chapter). In here, I also calculate amount of words and lemmas per act.

In [13]:
# directions per act
play_df["Directions per act"] = play_df["Amount of directions"]/play_df["Amount of acts"]
# words per act
play_df["Words per act"] = play_df["Words"]/play_df["Amount of acts"]
# lemmas per act
play_df["Lemmas per act"] = play_df["Lemmas"]/play_df["Amount of acts"]
play_df.head()

,Amount of acts,Amount of directions,Author,File,Lemmas,Lemmas per direction,Title,Words,Words per direction,Year,Directions per act,Words per act,Lemmas per act
0,1,38,"Пушкин, Александр Сергеевич",pushkin-stseny-iz-rytsarskih-vremen.xml,996,4.973684,Сцены из рыцарских времен,3399,3.131579,1837,38.000000,3399.0,996.000000
1,1,687,"Тургенев, Иван Сергеевич",turgenev-holostjak.xml,2785,6.391557,Холостяк,21501,4.506550,1849,687.000000,21501.0,2785.000000
2,2,254,"Гоголь, Николай Васильевич",gogol-zhenitba.xml,2187,5.952756,Женитьба,13094,3.925197,1842,127.000000,6547.0,1093.500000
3,3,88,"Крылов, Иван Андреевич",krylov-sonnyj-poroshok-ili-pohischennaja-krest...,1550,6.522727,Сонный порошок или похищенная крестьянка,7605,4.215909,1800,29.333333,2535.0,516.666667
4,1,80,"Блок, Александр Александрович",blok-neznakomka.xml,1342,18.025000,Незнакомка,4222,16.462500,1907,80.000000,4222.0,1342.000000


### Saving the dataframe

In [14]:
play_df.to_csv(csv_folder + os.sep + "general_information.csv", index=False, sep=";", encoding="utf-8")